In [1]:
source(file = "R/packages.R")
source(file = "R/utils.R")
source(file = "R/variables.R")
source(file = "R/mysql.R")

library(RMySQL)
library(ggplot2)
library(reshape)
library(sqldf)
library(digest)
library(xtable)

Loading required package: DBI
Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite

Attaching package: ‘RSQLite’

The following object is masked from ‘package:RMySQL’:

    isIdCurrent

sqldf will default to using MySQL


# DataSet Statistics

In [5]:
print("Running database queries...")
con <- dbConnect(dbDriver("MySQL"), user = mysql.user, password = mysql.passwd, 
                 dbname = mysql.db, host = mysql.host)

[1] "Running database queries..."


In [6]:
# Total repos
res <- dbSendQuery(con, "select count(*) as cnt from projects")
repos <- fetch(res, n = -1)
print(sprintf("Total repos: %d",repos$cnt))

[1] "Total repos: 64263412"


In [7]:
# Total users
res <- dbSendQuery(con, "select count(*) as cnt from users")
users <- fetch(res, n = -1)
print(sprintf("Total users: %d",users$cnt))

[1] "Total users: 19925203"


In [8]:
# Original repos
res <- dbSendQuery(con, "select count(*) as cnt from projects where forked_from is null and deleted is false and name not regexp '^.*\\.github\\.com$' and name <> 'try_git' and name <> 'dotfiles' and name <> 'vimfiles'")
orig_repos <- fetch(res, n = -1)
print(sprintf("Original repos: %f",orig_repos$cnt))

[1] "Original repos: 36419081.000000"


In [9]:
# % of original repos 
print(sprintf("Original repos: %f",(orig_repos$cnt/repos$cnt) * 100))


[1] "Original repos: 56.671565"


In [ ]:
# Original repositories that received a single commit in 2013
res <- dbSendQuery(con, "select count(*) as cnt from projects p where forked_from is null and deleted is false and name not regexp '^.*\\.github\\.com$' and name <> 'try_git' and name <> 'dotfiles' and name <> 'vimfiles' and exists ( select * from project_commits pc, commits c where  pc.project_id = p.id and  c.id = pc.commit_id and unix_timestamp(c.created_at) between 1359676800 and 1377993600)")
repos_with_commits <- fetch(res, n = -1)
print(sprintf("Original repos with commits Feb-Aug 2013: %f", repos_with_commits$cnt))
